# Project: Data Wrangling - [WeRateDogs]

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#ref">References</a></li>
</ul>

<a id='intro'></a>
## Introduction
> WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. These ratings almost always have a rating more than 10/10. Another data source is image prediction also there is another source about favorite count and retweet.
> But there is important question that needs good analysis to answer "Which is the favorite type?"
>
> Is it the most frequent type?
>
> Is it the type with highest average rating?
>
> Is it the type that has highest retweets and favorite counts?
>
> Can we depend on the first image only to determine the dog type?

<a id='wrangling'></a>
## Data Wrangling

## Loading the libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import json

## Gathering Data

### Twitter Archive 

In [2]:
# Import the twitter archive CSV file into a DataFrame
df_twitter = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
# Check to see if the file was imported correctly
df_twitter.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


### Image Prediciton

In [4]:
# Import the data using the given url 
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
response = requests.get(url)

with open('image-predictions.tsv',mode='wb') as file:
    file.write(response.content)

In [5]:
# Import the Image Predictions TSV file into a DataFrame
df_image = pd.read_csv('image-predictions.tsv',sep='\t')

In [6]:
# Check to see if the file was imported correctly
df_image.head(1)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True


### Twitter API

In [ ]:
import Tweepy
from Tweepy import OAuthHandler
import json
from timeit import default_timer as timer

# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# These are hidden to comply with Twitter's API terms and conditions
consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

# NOTE TO REVIEWER: this student had mobile verification issues so the following
# Twitter API code was sent to this student from a Udacity instructor
# Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = df_twitter.tweet_id.values
len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

In [7]:
# loading the text tile into list then json and afterthat data frame.
df_list = []

with open('tweet-json.txt') as file:
    for line in file:
        df_list.append(json.loads(line))
df_tweets = pd.DataFrame(df_list)

In [8]:
df_tweets.head(3)

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,source,in_reply_to_status_id,...,favorite_count,favorited,retweeted,possibly_sensitive,possibly_sensitive_appealable,lang,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status
0,Tue Aug 01 16:23:56 +0000 2017,892420643555336193,892420643555336193,This is Phineas. He's a mystical boy. Only eve...,False,"[0, 85]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892420639486877696, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,39467,False,False,False,False,en,NaN,NaN,NaN,NaN
1,Tue Aug 01 00:17:27 +0000 2017,892177421306343426,892177421306343426,This is Tilly. She's just checking pup on you....,False,"[0, 138]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892177413194625024, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,33819,False,False,False,False,en,NaN,NaN,NaN,NaN
2,Mon Jul 31 00:18:03 +0000 2017,891815181378084864,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,False,"[0, 121]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 891815175371796480, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,25461,False,False,False,False,en,NaN,NaN,NaN,NaN


In [9]:
# Write the column names to select the required columns
df_tweets.columns

Index(['created_at', 'id', 'id_str', 'full_text', 'truncated',
       'display_text_range', 'entities', 'extended_entities', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'retweet_count', 'favorite_count',
       'favorited', 'retweeted', 'possibly_sensitive',
       'possibly_sensitive_appealable', 'lang', 'retweeted_status',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status'],
      dtype='object')

In [10]:
df_tweets = pd.DataFrame(df_tweets,columns = ['id', 'retweet_count', 'favorite_count'])

In [11]:
# Check to see if the file was imported correctly
df_tweets.head()

,id,retweet_count,favorite_count
0,892420643555336193,8853,39467
1,892177421306343426,6514,33819
2,891815181378084864,4328,25461
3,891689557279858688,8964,42908
4,891327558926688256,9774,41048


## Assess

In [12]:
df_twitter.head(4)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None


#### From visual assessment:
##### Tidness issue:
        1- For name,doggo, floofer, pupper and puppo columns violate the rule of each variable makes a column.
        2- source column and expanded urls seems redundunt for each other violate the rule of each variable makes a column.
        3- Tweet_id is common column can be used to merge the two data frames for twitter and tweets .
        4- rating_denominator column is redundunt that we can remove and divide rating numerator by 10.
##### Quality issues:
        1-
        1-1- Missing values existed in column in_reply_to_status_id
        1-2- Missing values existed in column in_reply_to_user_id
        1-3- Missing values existed in column retweeted_status_id 
        1-4- Missing values existed in column retweeted_status_user_id
        1-5- Missing values existed in column retweeted_status_timestamp

##### For programatic assessment:

In [13]:
# Using info, descripe and value_counts for quick analysis of the missing values in dataframe
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   id              2354 non-null   int64
 1   retweet_count   2354 non-null   int64
 2   favorite_count  2354 non-null   int64
dtypes: int64(3)
memory usage: 55.3 KB


In [14]:
df_tweets.duplicated().sum()

0

In [15]:
df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

##### Quality issues:
        2- Wrong selection of data type for several columms like in the tweet_id and time stamp
        3- None in doggo, floofer, pupper, puppo columns is treated as a non-null value.
        4- some tweets are retweeted.

In [16]:
df_twitter.rating_denominator.value_counts()

10     2333
11        3
50        3
20        2
80        2
0         1
120       1
7         1
170       1
150       1
130       1
90        1
110       1
2         1
70        1
40        1
16        1
15        1
Name: rating_denominator, dtype: int64

##### Quality issues:
        5-1- Inacurate values existed in column rating_denominator that is not equal 10.

In [17]:
df_twitter.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
0         2
15        2
75        2
420       2
182       1
204       1
143       1
121       1
99        1
20        1
45        1
27        1
17        1
24        1
26        1
44        1
50        1
60        1
80        1
84        1
88        1
1776      1
960       1
666       1
144       1
165       1
Name: rating_numerator, dtype: int64

##### Quality issues:
        5-2- Inacurate values existed in column rating_numerator that is more than 20.

In [18]:
df_image.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [19]:
df_image.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [20]:
df_image.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


#### Quality Issues
    6- Column headers should be more descriptive (examples of non-descriptive column header are img_num, p1, p1_conf, p1_dog, etc.).
    7- Not all tweets have image prediction, only 2278 of 2356.    
    8- The name of the picture in image predicitions sometimes start with capital letters and another times with small letters.
    9- The tweets is not necessary for the analysis.

## Clean

In [21]:
df_tweets_clean = df_tweets.copy()
df_twitter_clean = df_twitter.copy()
df_image_clean = df_image.copy()

#### Quality Issues
    4- Some tweets are retweeted.

### Define

Keep only records that are null in retweet related columns

### Code

In [22]:
df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [23]:
df_twitter_clean = df_twitter_clean[df_twitter_clean['retweeted_status_id'].isnull()]

### Test

In [24]:
# Check that the columns related to the retweets are null which are retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp
df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2175 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2175 non-null   object 
 4   source                      2175 non-null   object 
 5   text                        2175 non-null   object 
 6   retweeted_status_id         0 non-null      float64
 7   retweeted_status_user_id    0 non-null      float64
 8   retweeted_status_timestamp  0 non-null      object 
 9   expanded_urls               2117 non-null   object 
 10  rating_numerator            2175 non-null   int64  
 11  rating_denominator          2175 non-null   int64  
 12  name                        2175 non-null   object 
 13  doggo                       2175 

### Missing Data

##### Quality issues:
        1-1- Missing values existed in column in_reply_to_status_id
        1-2- Missing values existed in column in_reply_to_user_id
        1-3- Missing values existed in column retweeted_status_id 
        1-4- Missing values existed in column retweeted_status_user_id
        1-5- Missing values existed in column retweeted_status_timestamp

### Define

Since these columns are almost null drop these columns.

### Code

In [25]:
df_twitter_clean.drop(columns=['in_reply_to_status_id', 'in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'],inplace=True,axis=1)

### Test

In [26]:
df_twitter_clean.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'doggo', 'floofer',
       'pupper', 'puppo'],
      dtype='object')

##### Quality issues:
        5-1- Inacurate values existed in column rating_denominator that is not equal 10.
##### Tidness issue:
        4- rating_denominator column is redundunt that we can remove and divide rating numerator by 10.

### Define

Drop rating_denominator from df_twitter_clean

### Code

In [27]:
df_twitter_clean.drop(columns='rating_denominator',inplace=True,axis=1)

### Test

In [28]:
assert 'rating_denominator' not in df_twitter_clean

##### Quality issues:
        5-2- Inacurate values existed in column rating_numerator that is more than 20.
##### Tidness issue:
        4- rating_denominator column is redundunt that we can remove and divide rating numerator by 10.

### Define

Filter the rows that has rating_numerator more than 20, then divide the rating numerator by 10

### Code

In [29]:
# Checking the size for rows removal step
df_twitter_clean.shape

(2175, 11)

In [30]:
# Removing rows with rating more than 20
df_twitter_clean.drop(inplace=True,index = df_twitter_clean[df_twitter_clean.rating_numerator > 20].index )

In [31]:
# Divide the rating numerator by 10
df_twitter_clean.rating_numerator /= 10

### Test

In [32]:
# If the number of rows decrease then the code works fine
df_twitter_clean.shape

(2152, 11)

In [33]:
# See the value of rating numerator column to enure that the divison step works correctly
df_twitter_clean.rating_numerator.value_counts()

1.2    500
1.0    442
1.1    426
1.3    307
0.9    156
0.8     98
0.7     54
1.4     43
0.5     36
0.6     32
0.3     19
0.4     17
0.2      9
0.1      8
0.0      2
1.5      1
1.7      1
2.0      1
Name: rating_numerator, dtype: int64

In [34]:
# Explore the common column for merging
all_columns = pd.Series(list(df_twitter) + list(df_tweets)+list(df_image))
all_columns[all_columns.duplicated()]

20    tweet_id
dtype: object

##### Quality issues:
    2- Wrong selection of data type for several columms like in the tweet_id and time stamp

### Define

Change the type of the tweet_it from integer to string in the three tables

### Code

In [35]:
df_tweets_clean.columns

Index(['id', 'retweet_count', 'favorite_count'], dtype='object')

In [36]:
df_twitter_clean.tweet_id=df_twitter_clean.tweet_id.astype(str);
df_tweets_clean.id =df_tweets_clean.id.astype(str);
df_image_clean.tweet_id=df_image_clean.tweet_id.astype(str);

### Test

In [37]:
print ('tweet id type in twitter table is',df_twitter_clean.tweet_id.dtype,'\n', 'tweet id type in tweets table is',
       df_tweets_clean.id.dtype,'\n','tweet id type in image table is', df_image_clean.tweet_id.dtype)

tweet id type in twitter table is object 
 tweet id type in tweets table is object 
 tweet id type in image table is object


##### Quality issues:
      3- None in doggo, floofer, pupper, puppo columns is treated as a non-null value.

### Define

Replace "None" value with correct None type for doggo, floofer, pupper, puppo

### Code

In [38]:
df_twitter_clean.replace('None',np.nan,inplace=True)

### Test

In [39]:
df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2152 entries, 0 to 2355
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tweet_id          2152 non-null   object 
 1   timestamp         2152 non-null   object 
 2   source            2152 non-null   object 
 3   text              2152 non-null   object 
 4   expanded_urls     2098 non-null   object 
 5   rating_numerator  2152 non-null   float64
 6   name              1488 non-null   object 
 7   doggo             87 non-null     object 
 8   floofer           10 non-null     object 
 9   pupper            233 non-null    object 
 10  puppo             25 non-null     object 
dtypes: float64(1), object(10)
memory usage: 201.8+ KB


#### Quality Issues
    6- Column headers should be more descriptive

### Define

Rename the column headers img_num, p1, p1_conf, p1_dog for the image table to be more descriptive

### Code

In [40]:
df_image_clean.columns

Index(['tweet_id', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2',
       'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'],
      dtype='object')

In [41]:
df_image_clean.rename(columns={"img_num": "image_number","p1":"Name_at_First_Picture","p1_conf":"Confindence_At_First_Algorithm",
                              "p2":"Name_at_Second_Picture","p2_conf":"Confindence_At_Second_Algorithm",
                              "p3":"Name_at_Third_Picture","p3_conf":"Confindence_At_Third_Algorithm"},inplace=True)

### Test

In [42]:
df_image_clean.columns

Index(['tweet_id', 'jpg_url', 'image_number', 'Name_at_First_Picture',
       'Confindence_At_First_Algorithm', 'p1_dog', 'Name_at_Second_Picture',
       'Confindence_At_Second_Algorithm', 'p2_dog', 'Name_at_Third_Picture',
       'Confindence_At_Third_Algorithm', 'p3_dog'],
      dtype='object')

#### Quality Issues
    7- Not all tweets have image prediction, only 2278 of 2356.

### Define

In [43]:
df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2152 entries, 0 to 2355
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tweet_id          2152 non-null   object 
 1   timestamp         2152 non-null   object 
 2   source            2152 non-null   object 
 3   text              2152 non-null   object 
 4   expanded_urls     2098 non-null   object 
 5   rating_numerator  2152 non-null   float64
 6   name              1488 non-null   object 
 7   doggo             87 non-null     object 
 8   floofer           10 non-null     object 
 9   pupper            233 non-null    object 
 10  puppo             25 non-null     object 
dtypes: float64(1), object(10)
memory usage: 201.8+ KB


### Code

In [44]:
df_twitter_clean = df_twitter_clean[df_twitter_clean['expanded_urls'].notnull()]

### Test

In [45]:
df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2098 entries, 0 to 2355
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tweet_id          2098 non-null   object 
 1   timestamp         2098 non-null   object 
 2   source            2098 non-null   object 
 3   text              2098 non-null   object 
 4   expanded_urls     2098 non-null   object 
 5   rating_numerator  2098 non-null   float64
 6   name              1488 non-null   object 
 7   doggo             84 non-null     object 
 8   floofer           10 non-null     object 
 9   pupper            231 non-null    object 
 10  puppo             24 non-null     object 
dtypes: float64(1), object(10)
memory usage: 196.7+ KB


#### Quality Issues
    8- The name of the picture in image predicitions sometimes start with capital letters and another times with small letters.

### Define

Change the start of each part of the dog name to be captalized 

### Code

In [46]:
df_image_clean.Name_at_First_Picture = df_image_clean.Name_at_First_Picture.str.title()
df_image_clean.Name_at_Second_Picture = df_image_clean.Name_at_Second_Picture.str.title()
df_image_clean.Name_at_Third_Picture = df_image_clean.Name_at_Third_Picture.str.title()

### Test

In [47]:
df_image_clean.sample(3)

,tweet_id,jpg_url,image_number,Name_at_First_Picture,Confindence_At_First_Algorithm,p1_dog,Name_at_Second_Picture,Confindence_At_Second_Algorithm,p2_dog,Name_at_Third_Picture,Confindence_At_Third_Algorithm,p3_dog
482,675372240448454658,https://pbs.twimg.com/media/CV9nd30XAAAEba5.jpg,1,Chihuahua,0.416385,True,West_Highland_White_Terrier,0.102933,True,Samoyed,0.087300,True
1434,773670353721753600,https://pbs.twimg.com/media/CryhFC0XEAA9wp_.jpg,1,Old_English_Sheepdog,0.969311,True,Maltese_Dog,0.013243,True,Soft-Coated_Wheaten_Terrier,0.004857,True
1716,819006400881917954,https://pbs.twimg.com/media/C12x-JTVIAAzdfl.jpg,4,Prison,0.907083,False,Palace,0.020089,False,Umbrella,0.007850,False


##### Tidness issue:
        1- For name,doggo, floofer, pupper and puppo columns violate the rule of each variable makes a column.

### Define

Extract the dogs types from the existed text then in one column called type with the value. Then drop the redundant columns.

### Code

In [48]:
df_twitter_clean['Type'] =  df_twitter_clean['text'].str.extract('(doggo|floofer|pupper|puppo)')

### Test

In [49]:
# To check the code effectiveness
df_twitter_clean.sample(2)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,name,doggo,floofer,pupper,puppo,Type
1859,675497103322386432,2015-12-12 02:07:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Reggie. He's going for the world record. ...,https://twitter.com/dog_rates/status/675497103...,1.1,Reggie,NaN,NaN,NaN,NaN,NaN
9,890240255349198849,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,https://twitter.com/dog_rates/status/890240255...,1.4,Cassie,doggo,NaN,NaN,NaN,doggo


### Code

In [50]:
# Drop the extra columns
df_twitter_clean.drop(columns=['doggo','floofer','pupper','puppo'],inplace=True,axis=1)

### Test

In [51]:
# Check the new existed columns
df_twitter_clean.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'name', 'Type'],
      dtype='object')

#### Quality Issues
    9- The tweets is not necessary for the analysis.

### Define

Drop the tweets column

### Code

In [52]:
df_twitter_clean.drop(columns='text',inplace=True,axis=1)

### Test

In [53]:
assert 'text' not in df_twitter_clean

### Define

##### Tidness issue:
        2- source column and expanded urls seems redundunt for each other.

Drop source column

### Code

In [54]:
df_twitter_clean.drop(columns='source',inplace=True,axis=1)

### Test

In [55]:
assert 'source' not in df_twitter_clean

##### Tidness issue:
        3- Tweet_id is common column can be used to merge the two data frames for twitter and tweets to have one observation unit not seperated at two tables.

### Define

Check the shape of each table and then merge the two tables, then rename the columns to be merged after that check again the size of the merged table.

### Code

In [56]:
# Check the size of both of the tables
print ('twitter_clean size:',df_twitter_clean.shape)
print ('tweets_clean size:',df_tweets_clean.shape)
print ('image_clean size:',df_image_clean.shape)

twitter_clean size: (2098, 6)
tweets_clean size: (2354, 3)
image_clean size: (2075, 12)


df_twitter_clean.tweet_id=df_twitter_clean.tweet_id.to_string();
df_tweets_clean.id =df_tweets_clean.id.to_string();
df_image_clean.tweet_id=df_image_clean.tweet_id.to_string();

In [57]:
df_tweets_clean.rename(columns={"id": "tweet_id"},inplace=True)

In [58]:
# Selecting the join type to be inner to avoid any existance of null in the original table of twitter since the tweets rows
#are more than the twitter archive data
df_twitter_clean = pd.merge(df_twitter_clean, df_tweets_clean,on='tweet_id', how='inner')

In [59]:
print ('twitter_clean size:',df_twitter_clean.shape)

twitter_clean size: (2098, 8)


In [60]:
# Selecting the join type to be inner to avoid any existance of null in the original table of twitter since the twitter archive
# rows are more than the image rows
df_twitter_clean = pd.merge(df_twitter_clean, df_image_clean,on='tweet_id', how='inner')

### Test

In [61]:
print ('twitter_clean size:',df_twitter_clean.shape)

twitter_clean size: (1975, 19)


##### Note that the number of rows is the less than the orginal one as it  is the common rows between the two tables, also the columns increased to the sum of their columns -2 as tweet_id is mutual column that is existed two times extra

## Storing the data

We have two main table one for twitter data with image retweets and favourite count while the other table is about image prediction

In [62]:
# For the twitter and tweets cleand data 
df_twitter_clean.to_csv('twitter_archive_master.csv',index = False)

<a id='ref'></a>
## References

#### To convert from list to json 
https://pythonexamples.org/python-list-to-json/#:~:text=To%20convert%20a%20Python%20List,and%20returns%20a%20JSON%20String.
#### Drop columns in pandas
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
#### Change type to string
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_string.html
#### Format the words in columns
https://pandas.pydata.org/docs/reference/api/pandas.Series.str.capitalize.html
#### Replace string with null value type
https://stackoverflow.com/questions/17097236/replace-invalid-values-with-none-in-pandas-dataframe
#### Save file as csv 
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html

## Thank You